In [16]:
'''
This notebook filters mapped PacBio LRS data to remove:
    polyadenylated transcripts,
    7SK transcripts,
    non-unique reads,
    and splicing intermeditates
'''

In [2]:
import os
import sys
import re
import glob

import pysam
import pybedtools
from pybedtools import BedTool

import numpy as np
import pandas as pd

from plotnine import *
import warnings
warnings.filterwarnings('ignore')

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # export pdfs with editable font types in Illustrator

In [92]:
# Read in data filenames and annotations used for filtering

samFiles = ['../0_mapped_data/1_untreated_RSII.sam',
            '../0_mapped_data/1_untreated_SQ.sam',
            '../0_mapped_data/2_untreated_RSII.sam',
            '../0_mapped_data/2_untreated_SQ.sam',
            '../0_mapped_data/3_DMSO_RSII.sam',
            '../0_mapped_data/3_DMSO_SQ.sam',
            '../0_mapped_data/4_DMSO_RSII.sam',
            '../0_mapped_data/4_DMSO_SQ.sam']

Rn7sk = '../annotation/files/7sk.bed'

introns = pd.read_csv('../annotation_files/mm10_VM20_introns.bed', delimiter = '\t', names =  ['chr', 'start', 'end', 'name', 'score', 'strand']) # annotation of all mm10 introns in BED6 format, downloaded from USCS table browser

In [19]:
# Create a bed file from introns bed file that contains just the nucleotide -1 to the 5SS
introns.loc[introns['strand'] == '+', 'fiveSS'] = introns['start']
introns.loc[introns['strand'] == '-', 'fiveSS'] = introns['end']
introns.loc[introns['strand'] == '+', 'newStart'] = introns['fiveSS'] - 1
introns.loc[introns['strand'] == '-', 'newStart'] = introns['fiveSS']
introns.loc[introns['strand'] == '+', 'newEnd'] = introns['fiveSS']
introns.loc[introns['strand'] == '-', 'newEnd'] = introns['fiveSS'] + 1

# convert coordinates back to integer values
introns['newStart'] = introns['newStart'].astype(np.int64)
introns['newEnd'] = introns['newEnd'].astype(np.int64)

# save as BED6 file
introns.to_csv('introns_5SS.bed', 
               sep = '\t', 
               index = False, 
               columns = ['chr', 'newStart', 'newEnd', 'name', 'score', 'strand'], 
               header = False)

# save as bedtool object for intersect
introns_5SS_bedtool = pybedtools.BedTool('introns_5SS.bed')

In [93]:
# Define a function to filter out polyadenylated reads from PacBio nascent RNA LRS data

def filter_polyA(mapped_reads_file):
    
    def append_id(mapped_reads_file):
        name, ext = os.path.splitext(mapped_reads_file)
        return "{name}_{id}{ext}".format(name=name, id='polyAfiltered', ext=ext)
    
    output = open(append_id(mapped_reads_file), 'w')
    #keep = [] # make an empty list of reads to keep
    
    with open(mapped_reads_file, 'r') as f:

        for line in f:
            line = line.strip('\n')
            col = line.split('\t')
            if col[0][0] == '@': # write header lines into output file
                output.write(line + '\n')
                continue
            cigar = col[5] # gets cigar string from SAM file

            if cigar.count('S') >= 1: # searches for S in cigar string indicating soft-clipped
                index = cigar.find('S') # finds position of S in cigar string
                if index <= 3: # if index is small then clipped bases are on the front of the read...should be polyT
                    length_clipped = int(cigar.split("S")[0]) # get position of first S in cigar string
                    clipped_bases = col[9][:length_clipped] # get sequence of clipped bases from start to first S
                    if  clipped_bases.count('T') >= 4 and \
                        clipped_bases.count('T')/len(clipped_bases) >=0.9: # if minimum 4 T's and T content is >90% of soft-clipped bases
                            continue # skip lines that have polyT at beginning of read
                    else:
                        output.write(line + '\n') # print lines that do not have polyT

                if (index + 1) == len(cigar): # if the S is at the end of the cigar string
                    m = re.search('[0-9]{0,3}(?=S)' , cigar) # find number before last S in cigar string
                    length_clipped = int(m.group(0)) #get length of clipped bases at end of read...should be polyA
                    clipped_bases = col[9][:(-1 * (length_clipped + 1)):-1] # get sequence of clipped bases before last S
                    if  clipped_bases.count('A') >= 4 and \
                        clipped_bases.count('A')/len(clipped_bases) >=0.9: # if minimum 4 A's and A content is >90% of soft-clipped bases

                            continue # do not write lines that have polyA at end of read
                    else:
                        output.write(line + '\n') # write lines that do not have polyA at the end
            else:
                output.write(line + '\n')

#     reads_bedtool = BedTool(keep)
#     reads_bedtool.saveas('test_filtered.bed')
#     return reads_bedtool
    f.close()
    output.close()

In [94]:
# Define a function for filtering splicing intermediates from each data file
def filter_splicing_intermediates(bed_file):
    
    def name_file_no_splicing_int(bed_file):
        name, ext = os.path.splitext(bed_file)
        return "{name}_{id}{ext}".format(name=name, id='no_splicing_int', ext=ext)
    
    def name_file_splicing_int(bed_file):
        name, ext = os.path.splitext(bed_file)
        return "{name}_{id}{ext}".format(name=name, id='splicing_int', ext=ext)
    
    # first open and reorder coordinates of bed file to put 3'end in position for intersection
    data = pd.read_csv(bed_file, delimiter = '\t', names =  ['chr', 'start', 'end', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts'])
    data.loc[data['strand'] == '+', 'threeEnd'] = data['end']
    data.loc[data['strand'] == '-', 'threeEnd'] = data['start']
    data.loc[data['strand'] == '+', 'fiveEnd'] = data['start']
    data.loc[data['strand'] == '-', 'fiveEnd'] = data['end']
    data.loc[data['strand'] == '+', 'newStart'] = data['threeEnd'] - 1
    data.loc[data['strand'] == '-', 'newStart'] = data['threeEnd']
    data.loc[data['strand'] == '+', 'newEnd'] = data['threeEnd']
    data.loc[data['strand'] == '-', 'newEnd'] = data['threeEnd'] + 1

    # convert coordinates back to integer values
    data['newStart'] = data['newStart'].astype(np.int64)
    data['newEnd'] = data['newEnd'].astype(np.int64)
    data['fiveEnd'] = data['fiveEnd'].astype(np.int64)
    data['threeEnd'] = data['threeEnd'].astype(np.int64)
    
    # save a temporary bed file with data 3'end coordinates
    data.to_csv('tmp.bed', 
               sep = '\t', 
               index = False, 
               columns = ['chr', 'newStart', 'newEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'start', 'end'], 
               header = False)
        
    # intersect data 3'end with intron 5'SS coordinates to get splicing intermediates and non-intermediates
    tmp_bedfile = open('tmp.bed')
    data_bedtool = pybedtools.BedTool(tmp_bedfile)
    intersect1 = data_bedtool.intersect(introns_5SS_bedtool, u = True).saveas('tmp_splicing_int.bed')
    
    tmp_bedfile = open('tmp.bed')
    data_bedtool = pybedtools.BedTool(tmp_bedfile)
    intersect2 = data_bedtool.intersect(introns_5SS_bedtool, v = True).saveas('tmp_no_splicing_int.bed')

    # reorder coordinates of data files
    data1 = pd.read_csv('tmp_splicing_int.bed', delimiter = '\t', names =  ['chr', 'newStart', 'newEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'start', 'end'])
    data1.to_csv(name_file_splicing_int(bed_file), 
               sep = '\t', 
               index = False, 
               columns = ['chr', 'start', 'end', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts'], 
               header = False)
    
    data2 = pd.read_csv('tmp_no_splicing_int.bed', delimiter = '\t', names =  ['chr', 'newStart', 'newEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts', 'start', 'end'])
    data2.to_csv(name_file_no_splicing_int(bed_file), 
               sep = '\t', 
               index = False, 
               columns = ['chr', 'start', 'end', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts'], 
               header = False)
    
    # clean up temp files
    os.remove('tmp.bed')
    os.remove('tmp_no_splicing_int.bed')
    os.remove('tmp_splicing_int.bed')

In [95]:
# Define a function for filtering non-unique readnames from each data file
def filter_nonunique_reads(bed_file):
    
    def name_unique_reads(bed_file):
        name, ext = os.path.splitext(bed_file)
        return "{name}_{id}{ext}".format(name=name, id='unique', ext=ext)
    
    # first open and reorder coordinates of bed file to put 3'end in position for intersection
    all_data = pd.read_csv(bed_file, delimiter = '\t', names =  ['chr', 'start', 'end', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts'])
    grouped = all_data.groupby(['name']).size().to_frame(name = 'count').reset_index()

    # get read names that are unique and filter to keep only reads which have name count == 1
    is_unique =  grouped['count'] == 1
    unique = grouped[is_unique]
    unique_names = pd.Series(unique['name'].values) # create a series of readnames that have occur only once

    data_is_unique = all_data['name'].isin(unique_names)
    data_unique = all_data[data_is_unique] # filter data for readnames that are unique
    
    # save unique reads to a new file
    data_unique.to_csv(name_unique_reads(bed_file), 
               sep = '\t', 
               index = False, 
               columns = ['chr', 'start', 'end', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts'], 
               header = False)

In [23]:
# Filter polyadenylated reads
for file in samFiles:
    filter_polyA(file)

In [24]:
# Convert SAM files to BAM  files for further filtering
SAM = []
for file in glob.glob('./*_polyAfiltered.sam'):
    SAM.append(file)

for samfile in SAM:
    name, ext = os.path.splitext(samfile)
    bamfile = "{name}{ext}".format(name=name, ext='.bam')
    pysam.view('-S', '-b', '-o', bamfile, samfile, catch_stdout=False)

In [25]:
# Filter 7SK reads from BAM files
BAM = []
for file in glob.glob('./*_polyAfiltered.bam'):
    BAM.append(file)

for bamfile in BAM:
    name, ext = os.path.splitext(bamfile)
    name_7sk = "{name}_{id}{ext}".format(name=name, id='7SK_only', ext=ext)
    name_no_7sk = "{name}_{id}{ext}".format(name=name, id='no_7SK', ext=ext)
    pysam.view('-b', '-L', Rn7sk, '-U', name_no_7sk, bamfile, '-o', name_7sk, catch_stdout=False)

In [26]:
# Sort and index BAM files
BAM = []
for file in glob.glob('./*_polyAfiltered_no_7SK.bam'):
    BAM.append(file)
    
for bamfile in BAM:
    name, ext = os.path.splitext(bamfile)
    bamfileSorted = "{name}_{id}{ext}".format(name=name, id='sorted', ext=ext)
    pysam.sort('-o', bamfileSorted, bamfile, catch_stdout=False)
    pysam.index(bamfileSorted, catch_stdout=False)

In [27]:
# Convert BAM files to BED12    
sortedBAM = []
for file in glob.glob('./*_polyAfiltered_no_7SK_sorted.bam'):
    sortedBAM.append(file)

for file in sortedBAM:
    name, ext = os.path.splitext(file)
    bedfile = "{name}{ext}".format(name=name, ext='.bed')
    
    bam_file = pybedtools.BedTool(file)
    bedFile = bam_file.bam_to_bed(bed12 = True).saveas(bedfile)

In [96]:
# Filter non-unique intermediates from BED12 files
BED = []
for file in glob.glob('./*_polyAfiltered_no_7SK_sorted.bed'):
    BED.append(file)
    
for file in BED:
    filter_nonunique_reads(file)

In [98]:
# Filter splicing intermediates from BED12 files
introns_5SS_bedtool = pybedtools.BedTool('introns_5SS.bed')

BED = []
for file in glob.glob('./*_polyAfiltered_no_7SK_sorted_unique.bed'):
    BED.append(file)

for file in BED:
    filter_splicing_intermediates(file)

In [99]:
# Optional: combine replicates into untreated and DMSO-treated files

untreated = []
dmso = []
for file in glob.glob('./*untreated*_polyAfiltered_no_7SK_sorted_unique_no_splicing_int.bed'):
    untreated.append(file)
for file in glob.glob('./*DMSO*_polyAfiltered_no_7SK_sorted_unique_no_splicing_int.bed'):
    dmso.append(file)
    
with open('untreated_combined.bed', 'w') as outfile:
    for fname in untreated:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
                
with open('dmso_combined.bed', 'w') as outfile:
    for fname in dmso:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [100]:
# Count the number of reads in each file along the way

input_count = []
for file in samFiles:
    samfile = pysam.AlignmentFile(file, "rb")
    count = samfile.count()
    input_count.append(count)
    
polyA_filtered_count = []
for file in glob.glob('./*_polyAfiltered.sam'):
    samfile = pysam.AlignmentFile(file, "rb")
    count = samfile.count()
    polyA_filtered_count.append(count)
    
    
no_7sk_count = []
for file in glob.glob('./*_polyAfiltered_no_7SK_sorted.bam'):
    bamfile = pysam.AlignmentFile(file, "rb")
    count = bamfile.count()
    no_7sk_count.append(count)
        
unique_reads_count = []
for file in glob.glob('./*_polyAfiltered_no_7SK_sorted_unique.bed'):
    count = len(open(file).readlines())
    unique_reads_count.append(count)
    
no_splicing_int_count = []  
for file in glob.glob('./*_polyAfiltered_no_7SK_sorted_unique_no_splicing_int.bed'):
    count = len(open(file).readlines())
    no_splicing_int_count.append(count)

In [105]:
# Make a table of read counts that are filtered at each step

counts_df = pd.DataFrame(list(zip(samFiles, input_count, polyA_filtered_count, no_7sk_count, unique_reads_count, no_splicing_int_count)), 
                        columns =['Sample', 'Mapped', 'PolyA', '7SK', 'Non-unique Reads', 'Splicing Intermediates'])

counts_df.to_csv('filtering_stats.csv', 
               sep = '\t', 
               index = True, 
               header = True)

In [106]:
# Add a row with column totals

# counts_df = pd.read_csv('filtering_stats.csv', delimiter = '\t', index_col = 0)
counts_df.loc['Total']= counts_df.sum()
counts_df['Sample']['Total'] = 'Total'
counts_df

# Print a report on the number of reads filtered at each step

mapped = counts_df['Mapped']['Total']
polyA = counts_df['PolyA']['Total']
sevenSK = counts_df['7SK']['Total']
non_unique = counts_df['Non-unique Reads']['Total']
spl_int = counts_df['Splicing Intermediates']['Total']

print('Number of mapped reads is: ' + str(mapped))
print('Percent of polyA reads filtered is: ' + str(((mapped-polyA)/mapped)*100))
print('Percent of 7SK reads filtered is: ' + str(((polyA-sevenSK)/mapped)*100))
print('Percent of non-unique reads filtered is: ' + str(((sevenSK-non_unique)/mapped)*100))
print('Percent of splicing intermediates reads filtered is: ' + str(((non_unique-spl_int)/mapped)*100))
print('Percent of total reads filtered is: ' + str(((mapped-non_unique)/mapped)*100))

Number of mapped reads is: 1155629
Percent of polyA reads filtered is: 1.67043229271678
Percent of 7SK reads filtered is: 24.880216747762475
Percent of non-unique reads filtered is: 14.23648939235689
Percent of splicing intermediates reads filtered is: 4.155572419868314
Percent of total reads filtered is: 40.787138432836144


/Users/Kirsten/Applications/anaconda3/envs/nanoCOP/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [107]:
counts_df

,Sample,Mapped,PolyA,7SK,Non-unique Reads,Splicing Intermediates
0,1_untreated_RSII.sam,21585,21389,16260,14994,14392
1,1_untreated_SQ.sam,246281,244321,177301,163876,158192
2,2_untreated_RSII.sam,42449,41894,33544,32497,29964
3,2_untreated_SQ.sam,273317,269003,225904,217732,199768
4,3_DMSO_RSII.sam,19878,19526,14867,9076,8566
5,3_DMSO_SQ.sam,268953,265119,177619,114676,108400
6,4_DMSO_RSII.sam,27220,26700,19595,13036,11815
7,4_DMSO_SQ.sam,255946,248373,183712,118394,105161
Total,Total,1155629,1136325,848802,684281,636258


In [ ]:
# Melt counts table from wide to long format for plotting
df = pd.melt(counts_df, id_vars=['Sample'], value_vars=['Mapped', 'PolyA', '7SK', 'Splicing Intermediates', 'Non-unique Reads'])

# add categorial variable to control the order of plotting
variable_cat = pd.Categorical(df['variable'], categories = ['Mapped',
                                                            'PolyA', 
                                                            '7SK',
                                                            'Non-unique Reads', 
                                                            'Splicing Intermediates'])

df = df.assign(variable_cat = variable_cat)

In [111]:
# plot count values across all samples
plt = (
    ggplot(aes(x = 'variable_cat', y = 'value', fill = 'variable'), df) + 
    geom_bar(stat = 'identity', position = 'dodge') + 
    facet_wrap('Sample', scales = 'free_y') +
    theme_classic() +
    theme(subplots_adjust={'wspace':0.8}) +
    theme(axis_text_x=element_text(rotation=45, hjust=1))
)
# plt
plt.save(filename = 'filtering_counts.pdf')

/Users/Kirsten/Applications/anaconda3/envs/nanoCOP/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
  from_inches(height, units), units), PlotnineWarning)
/Users/Kirsten/Applications/anaconda3/envs/nanoCOP/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: filtering_counts.pdf
  warn('Filename: {}'.format(filename), PlotnineWarning)
